In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
from basico import *
from process_bigraph import Composite, pf 
from biosimulator_processes.processes.copasi_process import CopasiProcess

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


### Example 1: Define a single process instance and a corresponding emitter

##### 1. Define the schema by which our Composite instance will be configured

In [4]:
instance = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model_file': 'biosimulator_processes/model_files/Caravagna2010.xml'
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    # TODO: Add emitter schema
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}

##### 2. Instantiate the Process-Bigraph-Engine with the Composite class

In [ ]:
workflow = Composite({
    'state': instance
})

##### 3. Run the Composite workflow with the Engine

In [ ]:
workflow.run(10)

##### 4. Gather and neatly-print the results.

In [ ]:
results = workflow.gather_results()

print(f'RESULTS: {pf(results)}')

##### 5. Plot/Analyze the results with Plotly

In [ ]:
# 5. Plot/Analyze the results

### Example 2: Define multiple processes in the same instance for a parameter scan